# Mutational Signature Extraction
In this notebook, we will extract SNV and indel signatures for the metastatic pheochromocytoma and paraganglioma cohort.

In [1]:
# Import required packages
import os
import csv
from SigProfilerMatrixGenerator.scripts import SigProfilerMatrixGeneratorFunc as matGen
from SigProfilerExtractor import sigpro as sig
import pandas as pd

In [2]:
# Set global variants
read_depth = 20
genome = 'GrCh38'
project = 'mPPGL'

## Create SigProfilerMatrixGenerator Input
We will use basic data wrangling to create the required input for SigProfilerMatrixGenerator

In [4]:
snvs = [file for file in os.listdir("../../data/raw/snvs/") if ".csv" in file] # Create list of SNV files
samples = [sample.split(".")[0] for sample in snvs] # Extract sample names

In [5]:
# Create input files
outfields = ['Project', 'Sample', 'ID', 'Genome', 'mut_type', 'chrom', 'pos_start', 'pos_end', 'ref', 'alt', 'type']

for snv, sample in zip(snvs, samples):
    with open("../../data/raw/snvs/{0}".format(snv), 'r') as infile, open("../../data/processed/mutational_signatures/input/{0}.SigProfiler.txt".format(sample), 'w') as outfile:
    
        reader = csv.DictReader(infile, delimiter=',')
        writer = csv.DictWriter(outfile, delimiter='\t', fieldnames=outfields, lineterminator='\r')
        writer.writeheader()

        for row in reader:

            if float(row['Tumor.AltDepth']) >= float(read_depth) and row['MUTECT2'] == "PASS":

                newDict = {'Project': project,
                        'Sample': row['Tumor.ID'],
                        'ID': '.', 
                        'Genome': genome,
                        'mut_type': row['Variant.Class'],
                        'chrom': row['Chr'],
                        'pos_start': row['Start'],
                        'pos_end': row['Start'], 
                        'ref': row['REF'],
                        'alt': row['ALT'],
                        'type': 'SOMATIC'}

                writer.writerow(newDict)

## Run SigProfilerMatrixGenerator

In [6]:
matrices = matGen.SigProfilerMatrixGeneratorFunc(project, 
                                                 genome, 
                                                 "../../data/processed/mutational_signatures/input",
                                                 plot=True, exome=False, bed_file=None, chrom_based=False, 
                                                 tsb_stat=True, seqInfo=True, cushion=100)

Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 248.64 seconds.
Starting matrix generation for INDELs...Completed! Elapsed time: 44.34 seconds.
Matrices generated for 44 samples with 846 errors. Total of 2122 SNVs, 19 DINUCs, and 748 INDELs were successfully analyzed.


## Extract SBS Signatures

In [7]:
# Load data
df1_path = "../../data/processed/mutational_signatures/input/output/SBS/mPPGL.SBS96.all"
df1 = pd.read_table(df1_path)
df1.head()

,MutationType,PP002-DZ3A,PP002-DZ4A,PP002-DZ5A,PP008-DZ2A,PP009-DZ1B,PP009-DZ2B,PP010-DZ1A,PP010-DZ5A,PP010-F2,...,PP253-DZ1A,PP253-DZ2A,PP263-DZ2A,PP302-DZ1A,PP323-DZ3A,PP323-DZ4A,PP323-DZ5A,PP341-DZ2B,PP344-DZ1A,PP344-DZ3A
0,A[C>A]A,0,0,0,3,0,0,0,0,0,...,1,0,0,0,1,17,4,0,0,1
1,A[C>A]C,0,0,0,3,0,0,0,0,0,...,0,0,0,2,0,9,3,0,0,2
2,A[C>A]G,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,7,2,0,0,0
3,A[C>A]T,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,12,0,0,0,0
4,A[C>G]A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0


In [8]:
# Run extraction
sig.sigProfilerExtractor(
        "matrix",
        "../../data/processed/mutational_signatures/SigProfilerExtractor_SBS",
        df1,
        opportunity_genome="GrCh38",
        exome=True,
        minimum_signatures=1,
        maximum_signatures=10,
        cpu=4,
    )


************** Reported Current Memory Use: 1.17 GB *****************

Extracting signature 1 for mutation type 96
The matrix normalizing cutoff is 9600


Time taken to collect 100 iterations for 1 signatures is 50.67 seconds
Optimization time is 2.821291923522949 seconds
The reconstruction error is 0.7388, average process stability is 1.0 and 
the minimum process stability is 1.0 for 1 signatures


Extracting signature 2 for mutation type 96
The matrix normalizing cutoff is 9600


Time taken to collect 100 iterations for 2 signatures is 68.21 seconds
Optimization time is 2.6764321327209473 seconds
The reconstruction error is 0.4993, average process stability is 0.96 and 
the minimum process stability is 0.95 for 2 signatures


Extracting signature 3 for mutation type 96
The matrix normalizing cutoff is 9600


Time taken to collect 100 iterations for 3 signatures is 123.71 seconds
Optimization time is 2.747195243835449 seconds
The reconstruction error is 0.2539, average process stabil

## Extract Indel Signatures

In [11]:
# Load data
df2_path = "../../data/processed/mutational_signatures/input/output/ID/mPPGL.ID83.all"
df2 = pd.read_table(df2_path)
df2.head()

,MutationType,PP002-DZ3A,PP002-DZ4A,PP002-DZ5A,PP008-DZ2A,PP009-DZ1B,PP009-DZ2B,PP010-DZ1A,PP010-DZ5A,PP010-F2,...,PP253-DZ1A,PP253-DZ2A,PP263-DZ2A,PP302-DZ1A,PP323-DZ3A,PP323-DZ4A,PP323-DZ5A,PP341-DZ2B,PP344-DZ1A,PP344-DZ3A
0,1:Del:C:0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1:Del:C:1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1:Del:C:2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1:Del:C:3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1:Del:C:4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
sig.sigProfilerExtractor(
        "matrix",
        "../../data/processed/mutational_signatures/SigProfilerExtractor_ID",
        df2,
        opportunity_genome="GrCh38",
        exome=True,
        minimum_signatures=1,
        maximum_signatures=10,
        cpu=4,
    )


************** Reported Current Memory Use: 1.53 GB *****************

Extracting signature 1 for mutation type INDEL
The matrix normalizing cutoff is 8300


Time taken to collect 100 iterations for 1 signatures is 76.49 seconds
Optimization time is 4.839684963226318 seconds
The reconstruction error is 0.0674, average process stability is 1.0 and 
the minimum process stability is 1.0 for 1 signatures


Extracting signature 2 for mutation type INDEL
The matrix normalizing cutoff is 8300


Time taken to collect 100 iterations for 2 signatures is 111.64 seconds
Optimization time is 4.449145793914795 seconds
The reconstruction error is 0.0535, average process stability is 0.85 and 
the minimum process stability is 0.69 for 2 signatures


Extracting signature 3 for mutation type INDEL
The matrix normalizing cutoff is 8300


Time taken to collect 100 iterations for 3 signatures is 155.08 seconds
Optimization time is 4.411947011947632 seconds
The reconstruction error is 0.0688, average proce